In [1]:
# ViT multi-class (9) classification
# Dataset: PathMNIST MedMNIST (Yang et al., 2021)
# Model: Transformer ViTForImageClassification

!python -c "import medmnist" || pip install -q medmnist
!python -c "import evaluate" || pip install -q evaluate --no-deps

import torch
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from transformers import ViTConfig, ViTForImageClassification, TrainingArguments, Trainer

import numpy as np
from PIL import Image
from medmnist import PathMNIST
from sklearn.metrics import accuracy_score, f1_score

# greyscale transform
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# load datasets
train_dataset = PathMNIST(split="train", download=True, transform=transform)
val_dataset = PathMNIST(split="val", download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'medmnist'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.5 MB/s eta 0:00:00


2025-09-29 20:11:39.551097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759176699.747032      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759176699.800834      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 206M/206M [00:07<00:00, 29.2MB/s]


In [2]:
# initialize model
config = ViTConfig(
    image_size=28,
    patch_size=7,
    num_channels=1,
    hidden_size=128,
    num_hidden_layers=4,
    num_attention_heads=4,
    num_labels=9
)
model = ViTForImageClassification(config)
model = model.to(device)

In [3]:
# training using standard transformer pipeline
def collate_fn(batch):
    images, labels = zip(*batch)
    return {
        "pixel_values": torch.stack(images),
        "labels": torch.tensor(labels).squeeze().long()
    }

training_args = TrainingArguments(
    output_dir="./model",
    save_strategy="no",
    learning_rate=3e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    num_train_epochs=10,
    report_to="none", # disable wandb for kaggle
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
)
trainer.train()

model.save_pretrained("./model")

/tmp/ipykernel_19/2084020293.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  "labels": torch.tensor(labels).squeeze().long()


Step,Training Loss
500,1.794200
1000,1.490500
1500,1.406900
2000,1.369600
2500,1.342800
3000,1.312500
3500,1.282400
4000,1.261800
4500,1.238100
5000,1.206500


In [4]:
import evaluate
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.9395607113838196,
 'eval_model_preparation_time': 0.0013,
 'eval_accuracy': 0.6499400239904038,
 'eval_f1': 0.6425615748102118,
 'eval_runtime': 2.0832,
 'eval_samples_per_second': 4802.291,
 'eval_steps_per_second': 75.366}